In [0]:
# Data Loading
df = spark.table("hackathon_cars_data")
display(df.limit(5))

vin back_legroom bed bed_height bed_length body_type cabin city city_fuel_economy combine_fuel_economy daysonmarket dealer_zip description engine_cylinders engine_displacement engine_type exterior_color fleet frame_damaged franchise_dealer franchise_make front_legroom fuel_tank_volume fuel_type has_accidents height highway_fuel_economy horsepower interior_color isCab is_certified is_cpo is_new is_oemcpo latitude length listed_date listing_color listing_id longitude main_picture_url major_options make_name maximum_seating mileage model_name owner_count power price salvage savings_amount seller_rating sp_id sp_name theft_title torque transmission transmission_display trimId trim_name vehicle_damage_category wheel_system wheel_system_display wheelbase width year JTMRFREV9JJ731732 37.2 in null null null SUV / Crossover null West Caldwell 22.0 null 29 7006 *All advertised prices are plus tax, title, dmv, dealer fees, and dealer installed options.***New Brakes Front***, ***All New Filters***, ***Recent Oil Change***, ***Navigation***, ***Sunroof / Moonroof / Roof***, ***Toyota Safety Sense-P***, ***Pre-Collision Sys***, ***Dynamic Radar Cruise Control***, ***Lane Departure Alert w/ Steering Assist***, AWD, Black w/Fabric Seat Trim, ABS brakes, Active Cruise Control, Alloy wheels, Blind Spot Monitor, Electronic Stability Control, Front dual zone A/C, Heated door mirrors, Height-Adjustable Power Liftgate, Illuminated entry, Low tire pressure warning, Power moonroof, Power Premium Extra Value Package, Radio: Entune Premium Audio with Navigation, Rear Backup Camera w/Monitor, Remote keyless entry, Roof Rails, Smart Key System, Traction control.CARFAX One-Owner. Clean CARFAX.Certified. Toyota Certified Used Vehicles Details:* Warranty Deductible: $50* Powertrain Limited Warranty: 84 Month/100,000 Mile (whichever comes first) from TCUV purchase date* Roadside Assistance* Transferable Warranty* Vehicle History* Limited Comprehensive Warranty: 12 Month/12,000 Mile (whichever comes first) from certified purchase date. Roadside Assistance for 1 Year* 160 Point Inspection2018 Toyota RAV4 XLE22/28 City/Highway MPG Awards:* 2018 KBB.com 10 Best SUVs Under $25,000 * 2018 KBB.com Best Resale Value Awards * 2018 KBB.com 10 Most Awarded BrandsWe pride ourselves in delivering a truly outstanding experience to our customers at all levels and across all departments. We work hard to ensure our customers are dealt with honestly, in a straightforward manner, professionally, and courteously - Paul Miller.[!@@Additional Info@@!]All Wheel Drive,Power Steering,ABS,4-Wheel Disc Brakes,Brake Assist,Aluminum Wheels,Tires - Front All-Season,Tires - Rear All-Season,Temporary Spare Tire,Sun/Moonroof,Generic Sun/Moonroof,Heated Mirrors,Power Mirror(s),Integrated Turn Signal Mirrors,Rear Defrost,Privacy Glass,Intermittent Wipers,Variable Speed Intermittent Wipers,Rear Spoiler,Power Door Locks,Fog Lamps,Daytime Running Lights,Automatic Headlights,AM/FM Stereo,CD Player,MP3 Player,Bluetooth Connection,Back-Up Camera,Auxiliary Audio Input,Steering Wheel Audio Controls,Pass-Through Rear Seat,Bucket Seats,Rear Bench Seat,Adjustable Steering Wheel,Trip Computer,Power Windows,Leather Steering Wheel,Keyless Entry,Power Door Locks,Remote Trunk Release,Cruise Control,Adaptive Cruise Control,Climate Control,Multi-Zone A/C,A/C,Cloth Seats,Driver Vanity Mirror,Passenger Vanity Mirror,Power Windows,Power Door Locks,Trip Computer,Engine Immobilizer,Traction Control,Stability Control,Traction Control,Front Side Air Bag,Lane Departure Warning,Lane Keeping Assist,Lane Departure Warning,Tire Pressure Monitor,Driver Air Bag,Passenger Air Bag,Front Head Air Bag,Rear Head Air Bag,Passenger Air Bag Sensor,Knee Air Bag,Child Safety Locks I4 2500.0 I4 Electric Storm Blue false false true Toyota 42.6 in 15.9 gal Gasoline false 65.9 in 28.0 176.0 Black false null true false true 40.8559 183.5 in 2020-08-11 BLUE 279008850 -74.3101 https://static.cargurus.com/images/forsale/2020/09/02/05/39/2018

In [0]:
# Environment Setup
import pandas as pd
import numpy as np
from datetime import datetime
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, IsolationForest
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from pyspark.sql.functions import *
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
mlflow.set_experiment("/FlipFinder-Pro")

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/3664698784130423', creation_time=1763009361634, experiment_id='3664698784130423', last_update_time=1763009361634, lifecycle_stage='active', name='/FlipFinder-Pro', tags={'mlflow.experiment.sourceName': '/FlipFinder-Pro',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'ujwal.kandi@utexas.edu',
 'mlflow.ownerId': '7753336194500419'}>

In [0]:
# Data Prep
pdf = df.toPandas()
print(f"Shape: {pdf.shape}")
print(f"Price range: ${pdf['price'].min():,.0f} - ${pdf['price'].max():,.0f}")
print(f"Year range: {pdf['year'].min()} - {pdf['year'].max()}")

critical_cols = ['price', 'year', 'mileage', 'make_name', 'model_name']
null_check = pdf[critical_cols].isnull().sum()

print(f"Nulls: {null_check.sum()}")

Shape: (20000, 66)
Price range: $1,495 - $199,995
Year range: 2000 - 2021
Nulls: 0


In [0]:
# Feature Engineering
pdf['has_accidents'] = pdf['has_accidents'].fillna(False).astype(bool)
pdf['age'] = 2024 - pdf['year']
pdf['mileage_per_year'] = pdf['mileage'] / (pdf['age'] + 1)
pdf['price_per_mile'] = pdf['price'] / (pdf['mileage'] + 1)
pdf['clean_title'] = (~pdf['has_accidents']).astype(int)

popular_makes = pdf['make_name'].value_counts().head(10).index

pdf['is_popular_make'] = pdf['make_name'].isin(popular_makes).astype(int)
pdf['single_owner'] = (pdf['owner_count'] == 1).astype(int)
pdf['daysonmarket'] = pdf['daysonmarket'].fillna(pdf['daysonmarket'].median())
pdf['combine_fuel_economy'] = pdf['combine_fuel_economy'].fillna(25)

In [0]:
# Market Analysis
market_stats = pdf.groupby(['make_name', 'model_name', 'year']).agg({
    'price': ['mean', 'median', 'count'],
    'daysonmarket': 'mean'
}).round(2)

market_stats.columns = ['_'.join(col).strip() for col in market_stats.columns]
market_stats.reset_index(inplace=True)

pdf = pdf.merge(
    market_stats[['make_name', 'model_name', 'year', 'price_median', 'daysonmarket_mean']],
    on=['make_name', 'model_name', 'year'],
    how='left'
)

pdf['market_value'] = pdf['price_median'].fillna(pdf['price'])
pdf['profit_potential'] = pdf['market_value'] - pdf['price']
pdf['profit_margin'] = (pdf['profit_potential'] / pdf['price'] * 100).round(2)

undervalued_count = (pdf['profit_margin'] > 10).sum()
print(f"Undervalued vehicles: {undervalued_count:,}")

Undervalued vehicles: 4,071


In [0]:
# ML Prep
Q1, Q3 = pdf['price'].quantile(0.25), pdf['price'].quantile(0.75)
IQR = Q3 - Q1
pdf_clean = pdf[(pdf['price'] >= Q1 - 1.5*IQR) & (pdf['price'] <= Q3 + 1.5*IQR)].copy()

pdf_clean['log_mileage'] = np.log1p(pdf_clean['mileage'])
pdf_clean['age_mileage_interaction'] = pdf_clean['age'] * pdf_clean['mileage'] / 10000

make_freq = pdf_clean['make_name'].value_counts() / len(pdf_clean)
model_freq = pdf_clean['model_name'].value_counts() / len(pdf_clean)
pdf_clean['make_frequency'] = pdf_clean['make_name'].map(make_freq)
pdf_clean['model_frequency'] = pdf_clean['model_name'].map(model_freq)

market_demand = pdf_clean.groupby(['make_name', 'model_name'])['daysonmarket'].median()
pdf_clean['market_demand_score'] = pdf_clean.apply(

    lambda x: market_demand.get((x['make_name'], x['model_name']), 45), axis=1
)

label_encoders = {}
for col in ['fuel_type', 'body_type', 'transmission']:
    if col in pdf_clean.columns:
        le = LabelEncoder()
        pdf_clean[f'{col}_encoded'] = le.fit_transform(pdf_clean[col].fillna('unknown'))
        label_encoders[col] = le

feature_columns = [
    'year', 'mileage', 'age', 'mileage_per_year', 'log_mileage',
    'age_mileage_interaction', 'daysonmarket', 'market_demand_score',
    'make_frequency', 'model_frequency', 'combine_fuel_economy', 'owner_count',
    'clean_title', 'is_popular_make', 'single_owner',
    'fuel_type_encoded', 'body_type_encoded', 'transmission_encoded'
]

feature_columns = [col for col in feature_columns if col in pdf_clean.columns]

X = pdf_clean[feature_columns].fillna(0)
y = pdf_clean['price']

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

pdf = pdf_clean.copy()

Train: 15,475 | Test: 3,869


In [0]:
# Model Training
models = {}

rf = RandomForestRegressor(n_estimators=200, max_depth=15, min_samples_split=5, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
rf_r2 = r2_score(y_test, rf.predict(X_test))
rf_mae = mean_absolute_error(y_test, rf.predict(X_test))
models['RandomForest'] = {'model': rf, 'r2': rf_r2, 'mae': rf_mae}

gb = GradientBoostingRegressor(n_estimators=150, max_depth=8, learning_rate=0.1, random_state=42)
gb.fit(X_train, y_train)
gb_r2 = r2_score(y_test, gb.predict(X_test))
gb_mae = mean_absolute_error(y_test, gb.predict(X_test))
models['GradientBoosting'] = {'model': gb, 'r2': gb_r2, 'mae': gb_mae}

et = ExtraTreesRegressor(n_estimators=200, max_depth=20, random_state=42, n_jobs=-1)
et.fit(X_train, y_train)
et_r2 = r2_score(y_test, et.predict(X_test))
et_mae = mean_absolute_error(y_test, et.predict(X_test))
models['ExtraTrees'] = {'model': et, 'r2': et_r2, 'mae': et_mae}

# FIX: Manual comparison instead of max() with key
best_r2 = 0
best_model_name = None
best_model = None
best_mae = None

for name, metrics in models.items():
    print(f"{name}: R²={metrics['r2']:.3f}, MAE=${metrics['mae']:,.0f}")
    if metrics['r2'] > best_r2:
        best_r2 = metrics['r2']
        best_model_name = name
        best_model = metrics['model']
        best_mae = metrics['mae']

print(f"\nBest: {best_model_name} ({best_r2*100:.1f}% accuracy)")

RandomForest: R²=0.877, MAE=$3,290
GradientBoosting: R²=0.896, MAE=$3,011
ExtraTrees: R²=0.872, MAE=$3,330

Best: GradientBoosting (89.6% accuracy)


In [0]:
# Scoring - Flip Score
X_all = pdf[feature_columns].fillna(0)
X_all_scaled = pd.DataFrame(scaler.transform(X_all), columns=X_all.columns, index=X_all.index)

pdf['predicted_value'] = best_model.predict(X_all_scaled)
pdf['flip_profit'] = pdf['predicted_value'] - pdf['price']
pdf['flip_margin'] = (pdf['flip_profit'] / pdf['price'] * 100).round(2)

pdf['flip_score'] = 50
pdf.loc[pdf['flip_margin'] > 30, 'flip_score'] += 40
pdf.loc[(pdf['flip_margin'] > 20) & (pdf['flip_margin'] <= 30), 'flip_score'] += 30
pdf.loc[(pdf['flip_margin'] > 10) & (pdf['flip_margin'] <= 20), 'flip_score'] += 20
pdf.loc[(pdf['flip_margin'] > 5) & (pdf['flip_margin'] <= 10), 'flip_score'] += 10

pdf.loc[pdf['daysonmarket'] < 20, 'flip_score'] += 20
pdf.loc[(pdf['daysonmarket'] >= 20) & (pdf['daysonmarket'] < 40), 'flip_score'] += 15
pdf.loc[(pdf['daysonmarket'] >= 40) & (pdf['daysonmarket'] < 60), 'flip_score'] += 10

pdf.loc[pdf['mileage'] < 30000, 'flip_score'] += 15
pdf.loc[(pdf['mileage'] >= 30000) & (pdf['mileage'] < 60000), 'flip_score'] += 10

pdf.loc[pdf['age'] <= 3, 'flip_score'] += 15
pdf.loc[(pdf['age'] > 3) & (pdf['age'] <= 5), 'flip_score'] += 10

pdf['flip_score'] = np.clip(pdf['flip_score'], 0, 100)

pdf['risk_level'] = 'Medium Risk'
pdf.loc[pdf['flip_score'] >= 80, 'risk_level'] = 'Hot Deal'
pdf.loc[(pdf['flip_score'] >= 65) & (pdf['flip_score'] < 80), 'risk_level'] = 'Low Risk'
pdf.loc[pdf['flip_score'] < 50, 'risk_level'] = 'High Risk'

pdf['confidence'] = np.clip(0.894 - np.abs(pdf['flip_profit'].values) / 200000, 0.60, 0.95)

print(f"Avg score: {pdf['flip_score'].mean():.1f}")
print(f"Hot deals: {(pdf['flip_score'] > 80).sum():,}")

Avg score: 82.1
Hot deals: 9,532


In [0]:
# Anomaly Detection
iso = IsolationForest(contamination=0.03, random_state=42, n_estimators=100)
anomaly_features = ['price', 'mileage', 'age', 'daysonmarket']
X_anomaly = pdf[anomaly_features].fillna(pdf[anomaly_features].median())

pdf['anomaly_score'] = iso.fit_predict(X_anomaly)
pdf['is_hidden_gem'] = (pdf['anomaly_score'] == -1) & (pdf['flip_profit'] > 5000)

hidden_gems = pdf[pdf['is_hidden_gem']].nlargest(10, 'flip_profit')
print(f"Hidden gems found: {pdf['is_hidden_gem'].sum()}")

if len(hidden_gems) > 0:

    display(hidden_gems[['make_name', 'model_name', 'year', 'price', 'flip_profit']])

Hidden gems found: 5


make_name,model_name,year,price,flip_profit
Lexus,ES 300,2001,3950.0,16816.39342370337
Volkswagen,Jetta Hybrid,2013,8975.0,14342.138791057805
RAM,ProMaster City,2019,21263.0,7354.357682925336
Subaru,Forester,2015,5999.0,6249.238681738105
Jaguar,XE,2019,46306.0,5069.548757452765


In [0]:
# Leaderboard
leaderboard = pdf.nlargest(20, 'flip_score')[
    ['make_name', 'model_name', 'year', 'price', 'predicted_value',
     'flip_profit', 'flip_margin', 'flip_score', 'risk_level',
     'daysonmarket', 'mileage', 'confidence']
].reset_index(drop=True)

leaderboard['rank'] = range(1, len(leaderboard) + 1)
leaderboard['vehicle'] = leaderboard['make_name'] + ' ' + leaderboard['model_name'] + ' (' + leaderboard['year'].astype(str) + ')'

spark_lb = spark.createDataFrame(leaderboard)
spark_lb.createOrReplaceTempView("flip_leaderboard")

display(leaderboard[['rank', 'vehicle', 'price', 'flip_profit', 'flip_score']].head(10))

rank,vehicle,price,flip_profit,flip_score
1,Toyota RAV4 (2017),19979.0,2154.74567825138,100
2,Hyundai Elantra (2020),21625.0,1358.9253665712968,100
3,Subaru Crosstrek (2021),29543.0,-1184.9392370165187,100
4,BMW X3 (2021),53695.0,-679.1901589074332,100
5,Hyundai Sonata (2021),24780.0,1168.0075806056411,100
6,Ford Fusion (2020),23748.0,1482.169926774386,100
7,Toyota 86 (2019),24695.0,2850.2658881895986,100
8,Honda Civic (2020),21255.0,1837.8762226742365,100
9,Ford Escape (2020),27172.0,3014.9782354365343,100
10,Subaru Outback (2019),26999.0,1869.3976911907812,100


In [0]:
# Visualizations
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Top Opportunities', 'Profit Distribution', 'Score vs Profit', 'Manufacturer Performance'),
    specs=[[{'type': 'bar'}, {'type': 'histogram'}],
           [{'type': 'scatter'}, {'type': 'bar'}]]
)

top10 = leaderboard.head(10)
fig.add_trace(
    go.Bar(y=top10['vehicle'][::-1], x=top10['flip_score'][::-1], 
           orientation='h', marker_color='lightblue', text=top10['flip_score'][::-1].round(1)),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x=pdf[pdf['flip_profit'] > -10000]['flip_profit'], nbinsx=30, marker_color='green'),
    row=1, col=2
)

# FIX: Replace min() with conditional
sample_size = 500 if len(pdf) > 500 else len(pdf)
sample = pdf.sample(sample_size)

fig.add_trace(
    go.Scatter(x=sample['flip_profit'], y=sample['flip_score'], mode='markers',
               marker=dict(size=5, color=sample['flip_score'], colorscale='Viridis')),
    row=2, col=1
)

mfg_scores = pdf.groupby('make_name')['flip_score'].mean().nlargest(10)
fig.add_trace(
    go.Bar(x=mfg_scores.index, y=mfg_scores.values, marker_color='coral'),
    row=2, col=2
)

fig.update_layout(height=700, showlegend=False, title_text="FlipFinder Analytics")
fig.show()

In [0]:
# Query System
def query_flips(q):
    q = q.lower()
    if 'best' in q or 'top' in q:
        return spark.sql("SELECT * FROM flip_leaderboard ORDER BY flip_score DESC LIMIT 5")
    elif 'toyota' in q:
        return spark.sql("SELECT * FROM flip_leaderboard WHERE LOWER(make_name) = 'toyota' LIMIT 5")
    elif 'profit' in q:
        return spark.sql("SELECT * FROM flip_leaderboard ORDER BY flip_profit DESC LIMIT 5")
    else:
        return spark.sql("SELECT * FROM flip_leaderboard LIMIT 5")

queries = ["Show best deals", "Toyota cars", "Highest profit"]
for q in queries:
    print(f"\n{q}:")
    query_flips(q).show(3, truncate=False)


Show best deals:
+---------+----------+----+-------+------------------+-------------------+-----------+----------+----------+------------+-------+------------------+----+-----------------------+
|make_name|model_name|year|price  |predicted_value   |flip_profit        |flip_margin|flip_score|risk_level|daysonmarket|mileage|confidence        |rank|vehicle                |
+---------+----------+----+-------+------------------+-------------------+-----------+----------+----------+------------+-------+------------------+----+-----------------------+
|Subaru   |Crosstrek |2021|29543.0|28358.06076298348 |-1184.9392370165187|-4.01      |100       |Hot Deal  |12          |1.0    |0.8880753038149174|3   |Subaru Crosstrek (2021)|
|Hyundai  |Elantra   |2020|21625.0|22983.925366571297|1358.9253665712968 |6.28       |100       |Hot Deal  |32          |12.0   |0.8872053731671435|2   |Hyundai Elantra (2020) |
|Toyota   |RAV4      |2017|19979.0|22133.74567825138 |2154.74567825138   |10.79      |100   

In [0]:
# Export
results = {
    'total_vehicles': len(pdf),
    'model_accuracy': f"{best_r2*100:.1f}%",
    'hot_deals': (pdf['flip_score'] > 80).sum(),
    'total_profit_opportunity': pdf[pdf['flip_profit'] > 0]['flip_profit'].sum(),
    'top_opportunity': f"{leaderboard.iloc[0]['vehicle']} - ${leaderboard.iloc[0]['flip_profit']:,.0f}"
}

for k, v in results.items():
    print(f"{k}: {v}")

leaderboard.to_csv('/tmp/flipfinder_results.csv', index=False)

total_vehicles: 19344
model_accuracy: 89.6%
hot_deals: 9532
total_profit_opportunity: 20145154.140591413
top_opportunity: Toyota RAV4 (2017) - $2,155


In [0]:
# ROI Analysis
def calculate_roi(capital=100000):
    affordable = leaderboard[leaderboard['price'] <= capital/5].head(5)
    investment = affordable['price'].sum()
    profit = affordable['flip_profit'].sum()
    days = affordable['daysonmarket'].mean()
    annual_roi = (profit / investment) * (365 / days) * 100   

    print(f"Capital: ${capital:,}")
    print(f"Vehicles: {len(affordable)}")
    print(f"Investment: ${investment:,.0f}")
    print(f"Profit: ${profit:,.0f}")
    print(f"ROI: {annual_roi:.1f}% annualized")
    print(f"vs Traditional: {annual_roi/25:.1f}x better")

    return affordable[['vehicle', 'price', 'flip_profit', 'flip_score']]

display(calculate_roi(100000))

Capital: $100,000
Vehicles: 5
Investment: $72,831
Profit: $12,016
ROI: 772.0% annualized
vs Traditional: 30.9x better


vehicle,price,flip_profit,flip_score
Toyota RAV4 (2017),19979.0,2154.74567825138,100
Mercedes-Benz C-Class (2007),4489.0,1348.0924952067526,100
Nissan Altima (2020),16900.0,4366.527108145041,100
Nissan Versa (2020),15464.0,2074.233597311668,100
Honda Accord (2017),15999.0,2072.2243299224465,100


In [0]:
# Dashboard Data Prep
dashboard_data = pdf[['make_name', 'model_name', 'year', 'price', 'predicted_value',
                      'flip_profit', 'flip_margin', 'flip_score', 'risk_level',
                      'daysonmarket', 'mileage', 'age', 'confidence']].copy()

dashboard_data['vehicle_full'] = dashboard_data['make_name'] + ' ' + dashboard_data['model_name']

spark_dashboard = spark.createDataFrame(dashboard_data)
spark_dashboard.write.mode('overwrite').saveAsTable('flipfinder_dashboard_data')

print(f"Dashboard table created: {len(dashboard_data):,} records")

Dashboard table created: 19,344 records


In [0]:
# Cell: Complete KPI Set for Dashboard
kpis = {
    'total_vehicles': len(pdf),
    'avg_flip_score': pdf['flip_score'].mean(),
    'total_profit_opportunity': pdf[pdf['flip_profit'] > 0]['flip_profit'].sum(),
    'hot_deals': (pdf['flip_score'] > 80).sum(),
    # NEW CRITICAL KPIs
    'model_accuracy': best_r2 * 100,  # KPI 5
    'avg_roi_pct': (leaderboard.head(20)['flip_profit'] / leaderboard.head(20)['price'] * 100).mean()  # KPI 6
}

print(f"📊 Vehicles Analyzed: {kpis['total_vehicles']:,}")
print(f"📈 Avg Flip Score: {kpis['avg_flip_score']:.1f}")
print(f"💰 Total Profit Opp: ${kpis['total_profit_opportunity']:,.0f}")
print(f"🔥 Hot Deals: {kpis['hot_deals']:,}")
print(f"🎯 Model Accuracy: {kpis['model_accuracy']:.1f}%")  # NEW
print(f"📈 Avg ROI: {kpis['avg_roi_pct']:.1f}%")  # NEW

📊 Vehicles Analyzed: 19,344
📈 Avg Flip Score: 82.1
💰 Total Profit Opp: $20,145,154
🔥 Hot Deals: 9,532
🎯 Model Accuracy: 89.6%
📈 Avg ROI: 11.8%
